# Day 20: Data Observability - Interactive Analysis

This notebook provides interactive analysis of the data observability system, allowing you to explore monitoring results, anomaly detection, and alerting patterns.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📊 Data Observability Analysis Environment Ready!")

## 1. Database Connection and Data Loading

In [ ]:
# Database connection
db_config = {
    'host': 'postgres',
    'port': '5432',
    'database': 'observability_db',
    'user': 'obs_user',
    'password': 'obs_password'
}

try:
    conn = psycopg2.connect(**db_config)
    print("✅ Connected to PostgreSQL database")
    
    # Load sample data
    transactions_df = pd.read_sql("""
        SELECT * FROM data_sources.customer_transactions 
        ORDER BY created_at DESC LIMIT 1000
    """, conn)
    
    metrics_df = pd.read_sql("""
        SELECT * FROM monitoring.data_quality_metrics 
        ORDER BY created_at DESC
    """, conn)
    
    alerts_df = pd.read_sql("""
        SELECT * FROM alerts.alert_history 
        ORDER BY created_at DESC
    """, conn)
    
    print(f"📊 Loaded {len(transactions_df)} transactions, {len(metrics_df)} metrics, {len(alerts_df)} alerts")
    
except Exception as e:
    print(f"❌ Database connection failed: {e}")
    print("💡 Make sure Docker containers are running: docker-compose up -d")

## 2. Data Quality Overview

In [ ]:
# Display basic statistics
print("📈 Transaction Data Overview:")
print(f"   • Total Transactions: {len(transactions_df):,}")
print(f"   • Date Range: {transactions_df['created_at'].min()} to {transactions_df['created_at'].max()}")
print(f"   • Unique Customers: {transactions_df['customer_id'].nunique():,}")
print(f"   • Total Amount: ${transactions_df['amount'].sum():,.2f}")

# Show data quality summary
print("\n🔍 Data Quality Summary:")
print(f"   • Missing Values: {transactions_df.isnull().sum().sum()}")
print(f"   • Duplicate Transactions: {transactions_df.duplicated().sum()}")
print(f"   • Status Distribution: {transactions_df['status'].value_counts().to_dict()}")

# Display sample data
transactions_df.head()

## 3. Five Pillars Analysis

In [ ]:
# Freshness Analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Data Observability - Five Pillars Analysis', fontsize=16, fontweight='bold')

# 1. Freshness - Time since last update
transactions_df['hours_since_created'] = (datetime.now() - pd.to_datetime(transactions_df['created_at'])).dt.total_seconds() / 3600
axes[0, 0].hist(transactions_df['hours_since_created'], bins=30, alpha=0.7, color='skyblue')
axes[0, 0].set_title('Data Freshness (Hours Since Creation)')
axes[0, 0].set_xlabel('Hours')
axes[0, 0].set_ylabel('Count')

# 2. Volume - Daily transaction counts
daily_counts = transactions_df.groupby(transactions_df['created_at'].dt.date).size()
axes[0, 1].plot(daily_counts.index, daily_counts.values, marker='o', color='green')
axes[0, 1].set_title('Data Volume (Daily Transaction Counts)')
axes[0, 1].set_xlabel('Date')
axes[0, 1].set_ylabel('Transaction Count')
axes[0, 1].tick_params(axis='x', rotation=45)

# 3. Distribution - Amount distribution
axes[1, 0].hist(transactions_df['amount'], bins=30, alpha=0.7, color='orange')
axes[1, 0].set_title('Data Distribution (Transaction Amounts)')
axes[1, 0].set_xlabel('Amount ($)')
axes[1, 0].set_ylabel('Frequency')

# 4. Schema - Status categories
status_counts = transactions_df['status'].value_counts()
axes[1, 1].pie(status_counts.values, labels=status_counts.index, autopct='%1.1f%%')
axes[1, 1].set_title('Data Schema (Status Distribution)')

plt.tight_layout()
plt.show()

print("📊 Five Pillars Analysis Complete!")

## 4. Anomaly Detection Simulation

In [ ]:
# Simulate anomaly detection on transaction amounts
amounts = transactions_df['amount'].values

# Calculate Z-scores
mean_amount = np.mean(amounts)
std_amount = np.std(amounts)
z_scores = (amounts - mean_amount) / std_amount

# Identify anomalies (|z-score| > 2)
anomaly_threshold = 2.0
anomalies = np.abs(z_scores) > anomaly_threshold
anomaly_indices = np.where(anomalies)[0]

# Create interactive plot
fig = go.Figure()

# Normal points
fig.add_trace(go.Scatter(
    x=np.arange(len(amounts))[~anomalies],
    y=amounts[~anomalies],
    mode='markers',
    name='Normal Transactions',
    marker=dict(color='blue', size=4, opacity=0.6)
))

# Anomaly points
if len(anomaly_indices) > 0:
    fig.add_trace(go.Scatter(
        x=anomaly_indices,
        y=amounts[anomaly_indices],
        mode='markers',
        name='Anomalies',
        marker=dict(color='red', size=8, symbol='x')
    ))

fig.update_layout(
    title='Anomaly Detection in Transaction Amounts',
    xaxis_title='Transaction Index',
    yaxis_title='Amount ($)',
    hovermode='closest'
)

fig.show()

print(f"🎯 Anomaly Detection Results:")
print(f"   • Total Transactions: {len(amounts):,}")
print(f"   • Anomalies Detected: {len(anomaly_indices)}")
print(f"   • Anomaly Rate: {len(anomaly_indices)/len(amounts)*100:.2f}%")
if len(anomaly_indices) > 0:
    print(f"   • Anomalous Amounts: {amounts[anomaly_indices]}")

## 5. Alert Pattern Analysis

In [ ]:
# Analyze alert patterns if we have alert data
if len(alerts_df) > 0:
    print("🚨 Alert Analysis:")
    
    # Alert severity distribution
    severity_counts = alerts_df['severity'].value_counts()
    print(f"   • Severity Distribution: {severity_counts.to_dict()}")
    
    # Alert status distribution
    status_counts = alerts_df['status'].value_counts()
    print(f"   • Status Distribution: {status_counts.to_dict()}")
    
    # Create alert timeline
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    # Severity over time
    severity_colors = {'low': 'green', 'medium': 'orange', 'high': 'red', 'critical': 'darkred'}
    for severity in alerts_df['severity'].unique():
        severity_data = alerts_df[alerts_df['severity'] == severity]
        ax1.scatter(pd.to_datetime(severity_data['created_at']), 
                   severity_data['current_value'], 
                   c=severity_colors.get(severity, 'blue'), 
                   label=severity, alpha=0.7, s=50)
    
    ax1.set_title('Alert Timeline by Severity')
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Alert Value')
    ax1.legend()
    ax1.tick_params(axis='x', rotation=45)
    
    # Alert frequency by metric
    metric_counts = alerts_df['metric_name'].value_counts()
    ax2.bar(metric_counts.index, metric_counts.values, color='skyblue')
    ax2.set_title('Alert Frequency by Metric')
    ax2.set_xlabel('Metric Name')
    ax2.set_ylabel('Alert Count')
    ax2.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
else:
    print("ℹ️ No alert data available for analysis")
    
    # Create sample alert simulation
    print("🎭 Simulating Alert Patterns...")
    
    # Generate sample alert data
    sample_alerts = pd.DataFrame({
        'metric_name': ['data_freshness', 'data_volume', 'schema_change', 'distribution_anomaly'] * 5,
        'severity': np.random.choice(['low', 'medium', 'high'], 20),
        'created_at': pd.date_range(start='2024-01-01', periods=20, freq='H'),
        'current_value': np.random.uniform(0.5, 1.0, 20)
    })
    
    # Plot simulated alerts
    fig = px.scatter(sample_alerts, 
                     x='created_at', 
                     y='current_value',
                     color='severity',
                     size='current_value',
                     hover_data=['metric_name'],
                     title='Simulated Alert Timeline')
    fig.show()
    
    print("📊 Simulated alert patterns displayed")

## 6. Data Health Score Calculation

In [ ]:
# Calculate comprehensive data health score
def calculate_data_health_score(df):
    """Calculate overall data health score based on multiple factors"""
    
    scores = {}
    
    # 1. Freshness Score (based on recency)
    hours_since_latest = (datetime.now() - pd.to_datetime(df['created_at']).max()).total_seconds() / 3600
    freshness_score = max(0, 1 - (hours_since_latest / 24))  # Degrade over 24 hours
    scores['freshness'] = freshness_score
    
    # 2. Volume Score (based on expected volume)
    expected_daily_volume = 1000  # Expected transactions per day
    actual_daily_volume = len(df) / ((pd.to_datetime(df['created_at']).max() - pd.to_datetime(df['created_at']).min()).days + 1)
    volume_score = min(1.0, actual_daily_volume / expected_daily_volume)
    scores['volume'] = volume_score
    
    # 3. Completeness Score (based on missing values)
    completeness_score = 1 - (df.isnull().sum().sum() / (len(df) * len(df.columns)))
    scores['completeness'] = completeness_score
    
    # 4. Consistency Score (based on data types and formats)
    consistency_score = 1.0  # Simplified - assume consistent
    scores['consistency'] = consistency_score
    
    # 5. Validity Score (based on business rules)
    valid_amounts = (df['amount'] > 0) & (df['amount'] < 10000)  # Reasonable amount range
    valid_quantities = (df['quantity'] > 0) & (df['quantity'] <= 100)  # Reasonable quantity range
    validity_score = (valid_amounts & valid_quantities).mean()
    scores['validity'] = validity_score
    
    # Calculate weighted overall score
    weights = {
        'freshness': 0.25,
        'volume': 0.20,
        'completeness': 0.20,
        'consistency': 0.15,
        'validity': 0.20
    }
    
    overall_score = sum(scores[metric] * weights[metric] for metric in weights)
    scores['overall'] = overall_score
    
    return scores

# Calculate health scores
health_scores = calculate_data_health_score(transactions_df)

# Display results
print("🏥 Data Health Assessment:")
print("=" * 40)
for metric, score in health_scores.items():
    status = "✅ Excellent" if score >= 0.9 else "🟡 Good" if score >= 0.7 else "🟠 Fair" if score >= 0.5 else "🔴 Poor"
    print(f"   • {metric.capitalize()}: {score:.3f} {status}")

# Create health score visualization
fig = go.Figure()

metrics = list(health_scores.keys())[:-1]  # Exclude overall
values = [health_scores[metric] for metric in metrics]

fig.add_trace(go.Scatterpolar(
    r=values,
    theta=metrics,
    fill='toself',
    name='Data Health Scores'
))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1]
        )
    ),
    title="Data Health Score Radar Chart",
    showlegend=True
)

fig.show()

print(f"\n🎯 Overall Data Health Score: {health_scores['overall']:.3f}")

## 7. Recommendations and Next Steps

In [ ]:
# Generate recommendations based on analysis
def generate_recommendations(health_scores, anomaly_count, alert_count):
    """Generate actionable recommendations based on analysis results"""
    
    recommendations = []
    
    # Health score based recommendations
    if health_scores['freshness'] < 0.8:
        recommendations.append({
            'priority': 'High',
            'category': 'Freshness',
            'issue': 'Data freshness below optimal threshold',
            'action': 'Increase data ingestion frequency or optimize ETL pipelines'
        })
    
    if health_scores['volume'] < 0.7:
        recommendations.append({
            'priority': 'Medium',
            'category': 'Volume',
            'issue': 'Data volume lower than expected',
            'action': 'Investigate data source issues or validate filtering logic'
        })
    
    if health_scores['completeness'] < 0.9:
        recommendations.append({
            'priority': 'High',
            'category': 'Completeness',
            'issue': 'Missing data detected',
            'action': 'Implement data validation rules and improve data collection processes'
        })
    
    if health_scores['validity'] < 0.8:
        recommendations.append({
            'priority': 'Medium',
            'category': 'Validity',
            'issue': 'Invalid data values detected',
            'action': 'Strengthen data validation rules and implement business logic checks'
        })
    
    # Anomaly based recommendations
    if anomaly_count > len(transactions_df) * 0.05:  # More than 5% anomalies
        recommendations.append({
            'priority': 'High',
            'category': 'Anomalies',
            'issue': f'High anomaly rate detected ({anomaly_count} anomalies)',
            'action': 'Review anomaly detection thresholds and investigate root causes'
        })
    
    # General recommendations
    if health_scores['overall'] >= 0.9:
        recommendations.append({
            'priority': 'Low',
            'category': 'Optimization',
            'issue': 'Data quality is excellent',
            'action': 'Continue monitoring and consider advanced analytics opportunities'
        })
    
    return recommendations

# Generate recommendations
recommendations = generate_recommendations(health_scores, len(anomaly_indices), len(alerts_df))

# Display recommendations
print("💡 Data Observability Recommendations:")
print("=" * 50)

if recommendations:
    for i, rec in enumerate(recommendations, 1):
        priority_emoji = {'High': '🔴', 'Medium': '🟡', 'Low': '🟢'}
        emoji = priority_emoji.get(rec['priority'], '🔵')
        
        print(f"{i}. {emoji} {rec['category']} ({rec['priority']} Priority)")
        print(f"   Issue: {rec['issue']}")
        print(f"   Action: {rec['action']}")
        print()
else:
    print("✅ No specific recommendations - data quality is optimal!")

print("\n🎯 Next Steps:")
print("   1. Set up automated monitoring alerts")
print("   2. Implement data quality rules in production")
print("   3. Create executive dashboards for stakeholders")
print("   4. Establish data SLA monitoring")
print("   5. Build automated remediation workflows")

print("\n🔗 Resources:")
print("   • Grafana Dashboards: http://localhost:3000")
print("   • Prometheus Metrics: http://localhost:9090")
print("   • Database Access: psql -h postgres -U obs_user observability_db")

In [ ]:
# Clean up database connection
if 'conn' in locals():
    conn.close()
    print("🔌 Database connection closed")

print("\n🎉 Data Observability Analysis Complete!")
print("Thank you for exploring the observability system.")